In [ ]:
%matplotlib inline
%load_ext rpy2.ipython
import vdmlab as vdm
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from load_data import assign_label, load_biconditional_events_general, vdm_assign_label
from core import Rat, combine_rats
from plotting import plot_behavior

from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

In [ ]:
pandas2ri.activate()
lme4 = importr('lme4')
car = importr('car')
lmerTest = importr('lmerTest')

In [ ]:
def evaluate_training(learning_df):
    ro.globalenv['rlearn'] = pandas2ri.py2ri(learning_df)
    ro.r("""rlearn <- within(rlearn, {
      trial <- factor(trial_type)
      rewarded <- factor(rewarded)
      measure <- factor(measure)
      value <- as.numeric(value)})""")

    ro.r("contrasts(rlearn$trial) <- c(3, 1, -1, -3)")
    ro.r("contrasts(rlearn$rewarded) <- c(1, -1)")

    ro.r("lmm <- lmer(value ~ rewarded * session + (1|measure), data=rlearn, REML=FALSE)")
    anova = ro.r("print(Anova(lmm,type=3))")
    summary = ro.r("print(summary(lmm))")
    return anova, summary

In [ ]:
# home = os.path.expanduser("~")
# root = os.path.join(home, 'code', 'emi_biconditional')
# data_filepath = os.path.join(root, 'cache', 'data')

data_filepath = os.path.join('E:\\', 'code', 'emi_biconditional', 'cache', 'data', 'winter2017')

In [ ]:
rats = ['R120', 'R121', 'R118', 'R119', 'R116', 'R117', 'R114']

groups = [1, 2, 1, 2, 1, 2, 1]

group1 = ['R120', 'R118', 'R116', 'R114']
group2 = ['R121', 'R119', 'R117']

# this_rat = 'R119'

# filename = os.path.join(data_filepath, '!' + session_id)

magazine_session = '!2017-01-17'

sessions = []
for file in sorted(os.listdir(data_filepath)):
    if file != magazine_session and file[0] == '!':
        sessions.append(os.path.join(data_filepath, file))

data = dict()
for rat in rats:
    data[rat] = Rat(rat, group1, group2)

for session in sessions:
    rats_data = vdm.load_medpc(os.path.join(data_filepath, session), assign_label)

    for rat, group in zip(rats, groups):
        data[rat].add_session(**rats_data[rat], group=group)

n_sessions = len(data[rats[0]].sessions)

df = combine_rats(data, rats, n_sessions)

In [ ]:
df.head()

In [ ]:
this_rat = 'R119'

In [ ]:
stats_df = df[df['cue'] == 'sound']
stats_df = stats_df[stats_df['rat'] == this_rat]
stats_df.head()

In [ ]:
anova, summary = evaluate_training(stats_df)

In [ ]:
anova

In [ ]:
summary